### Base de Datos(Database)

#### Objetivos
 1. Documentación sobre Spark SQL
2. Crear la Base de Datos "demo"
3. Acceder al **"Catalog"** en la "Interfaz de Usuario"
 4. Comando **"SHOW"**
5. Comando **DESCRIBE(DESC)**
6. Mostrar la Base de Datos Actual

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS demo;

In [0]:
%sql
SHOW DATABASES;


In [0]:
%sql
DESCRIBE DATABASE demo;

In [0]:
%sql
DESCRIBE DATABASE EXTENDED demo;

In [0]:
SELECT current_database();

In [0]:
SHOW TABLES;

In [0]:
SHOW TABLES IN demo;

In [0]:
USE demo;
SELECT current_database();

###  Tablas Administradas(Managed Tables)

#### Objetivos

1. Crear una **"Tabla Adminstrada(Managed Table)"** con Python
 2. Crear una **"Tabla Adminstrada(Managed Table)"** con SQL
 3. Efecto de eliminar una Tabla Administrada
 4. Describir(Describe) la Tabla

In [0]:
%run "../includes/configuration"

In [0]:
%python
results_movie_genre_language = spark.read.parquet(f"{gold_folder_path}/results_movie_genre_language")

In [0]:
%python
results_movie_genre_language.write.format("parquet").saveAsTable("demo.results_movie_genre_language_python")

In [0]:
%python
display(results_movie_genre_language)

In [0]:
USE demo;
SHOW TABLES;

Una tabla administrada en Azure Databricks es una tabla cuyo almacenamiento subyacente de archivos de datos es gestionado por Databricks junto con el registro en el metastore. Databricks recomienda usar tablas administradas siempre que se cree una nueva tabla. Las tablas administradas por Unity Catalog son el tipo de tabla predeterminado cuando se crean tablas en Azure Databricks y siempre utilizan Delta Lake

In [0]:
DESCRIBE EXTENDED results_movie_genre_language_python;

In [0]:
CREATE TABLE demo.results_movie_genre_language_sql
AS
SELECT *
FROM results_movie_genre_language_python
WHERE genre_name = 'Adventure'

In [0]:
SELECT * FROM results_movie_genre_language_sql

In [0]:
SELECT current_database();

In [0]:
DESCRIBE EXTENDED results_movie_genre_language_sql;

In [0]:
DROP TABLE IF EXISTS results_movie_genre_language_sql;

In [0]:
DROP TABLE IF EXISTS results_movie_genre_language_python;

In [0]:
SHOW TABLES IN demo;

Cuando se elimina una tabla adminsitrada se borran tanto los metadatos como los datos

### Tablas Externas(External Tables)

 #### Objetivos
  1. Crear una **"Tabla Externa(External Table)"** con Python
  2. Crear una **"Tabla Externa(External Table)"** con SQL
  3. Efecto de eliminación de una **"Tabla Externa(External Table)"**
  4. Describir(Describe) la Tabla

In [0]:
%python
spark.conf.set("spark.sql.legacy.allowNonEmptyLocationInCTAS", "true")

results_movie_genre_language.write.format("parquet").option(
    "path", f"{gold_folder_path}/results_movie_genre_language_py"
).saveAsTable("demo.results_movie_genre_language_py")

In [0]:
DESC EXTENDED demo.results_movie_genre_language_py


In [0]:
CREATE TABLE demo.results_movie_genre_language_sql(
  title STRING,
  duration_time INT,
  release_date DATE,
  vote_average FLOAT,
  language_name STRING,
  genre_name STRING,
  created_date TIMESTAMP
)
USING PARQUET
LOCATION "/mnt/moviehistory/gold/results_movie_genre_language_ext_sql"


In [0]:
SHOW TABLES IN demo;

In [0]:
INSERT INTO demo.results_movie_genre_language_sql
SELECT * FROM demo.results_movie_genre_language_py
WHERE genre_name = 'Adventure'

In [0]:
SELECT COUNT(1)
FROM demo.results_movie_genre_language_sql

In [0]:
SHOW TABLES IN demo;

In [0]:
DROP TABLE demo.results_movie_genre_language_sql

Al eliminar una tabla externa los datos no se eliminan de la ubicacion externa, si borra los metadatos

### Vistas(Views)

#### Objetivos
1. Crear Vista Temporal
2. Crear Vista Temporal Global
3. Crear Vista Permanente

Las vistas no ocupan espacio de almacenamiento, sirven para representar una consulta compleja o recurrente, funcionan como una tabla virtual.

In [0]:
SELECT current_database();

In [0]:
CREATE OR REPLACE TEMP VIEW v_results_movies_genres_language
AS
SELECT *
FROM demo.results_movie_genre_language_py
WHERE genre_name = 'Adventure';

In [0]:
SELECT * FROM v_results_movies_genres_language

In [0]:
CREATE OR REPLACE GLOBAL TEMP VIEW gv_results_movies_genres_language
AS
SELECT *
FROM demo.results_movie_genre_language_py
WHERE genre_name = 'Drama';

In [0]:
SHOW TABLES IN global_temp;

In [0]:
SELECT * FROM global_temp.gv_results_movies_genres_language;

Vista permanente - Despues de desconectarse del cluster, la vista seguira siendo accesible

In [0]:
CREATE OR REPLACE VIEW pv_results_movies_genres_language
AS
SELECT *
FROM demo.results_movie_genre_language_py
WHERE genre_name = 'Comedy';

In [0]:
SHOW TABLES;

In [0]:
SELECT * FROM pv_results_movies_genres_language;